In this code, I attempt to build a decision tree using the id3 algorithm on a custom dataset

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [78]:
data = pd.read_csv('/content/id3.csv')

This is our dataset with 14 columns

In [79]:
data

,Day,Outlook,Temperature,Humidity,Wind,Label
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


Storing the unique values in all columns.

In [80]:
col_vals={}
data = data.drop('Day', axis=1)
for column in data.columns:
    unique_values = data[column].unique().tolist()
    col_vals[column] = unique_values
col_vals

{'Outlook': ['Sunny', 'Overcast', 'Rain'],
 'Temperature': ['Hot', 'Mild', 'Cool'],
 'Humidity': ['High', 'Normal'],
 'Wind': ['Weak', 'Strong'],
 'Label': ['No', 'Yes']}

The total entropy of the dataset has been calculated using the formula total_entropy = - (prob_yes * math.log2(prob_yes) + prob_no * math.log2(prob_no))

In [54]:
label_vals = data['Label'].value_counts()

total_samples = len(data)
prob_yes = label_vals.get('Yes') / total_samples
prob_no = label_vals.get('No') / total_samples

if prob_yes == 0 or prob_no == 0:
    total_entropy = 0
else:
    total_entropy = - (prob_yes * math.log2(prob_yes) + prob_no * math.log2(prob_no))
total_entropy = round(total_entropy, 2)
print("Total Entropy:", total_entropy)

Total Entropy: 0.94


This is the entropy function for calculating the entropy of each feature of each attribute and this will be used to calculate the information gain of each attribute

In [55]:
def entropy(data, column_name):
    value_counts = data[column_name].value_counts()
    total_samples = len(data)
    probabilities = value_counts / total_samples
    entropy_value = -(probabilities * probabilities.apply(math.log2)).sum()
    return entropy_value

This is the function for information gain. Information gain is the weighted entropy subtracted from the total entropy of the dataset calculated above.

In [56]:
def information_gain(data, feature_name, class_name):
    total_entropy = entropy(data, class_name)
    unique_values = data[feature_name].unique()
    info_gain = total_entropy

    for value in unique_values:
        subset = data[data[feature_name] == value]
        subset_entropy = entropy(subset, class_name)
        subset_weight = len(subset) / len(data)
        info_gain -= subset_weight * subset_entropy
    return info_gain

Here, we have printed the entropies of each feature

In [57]:
entropies = {}

for column_name in data.columns:
    if column_name != 'Label':
        feature_entropies = {}

        for feature_value in data[column_name].unique():
            subset = data[data[column_name] == feature_value]
            feature_entropy = entropy(subset, 'Label')
            feature_entropies[feature_value] = feature_entropy

        entropies[column_name] = feature_entropies

print(entropies)

{'Outlook': {'Sunny': 0.9709505944546686, 'Overcast': -0.0, 'Rain': 0.9709505944546686}, 'Temperature': {'Hot': 1.0, 'Mild': 0.9182958340544896, 'Cool': 0.8112781244591328}, 'Humidity': {'High': 0.9852281360342515, 'Normal': 0.5916727785823275}, 'Wind': {'Weak': 0.8112781244591328, 'Strong': 1.0}}


Information gains of each column have been printed whioch are being used for the root node selection

In [58]:
information_gains = {}
for column_name in data.columns:
    if column_name != 'Label':
        feature_information_gain = information_gain(data, column_name, 'Label')
        information_gains[column_name] = feature_information_gain

print("Information Gains:", information_gains)

Information Gains: {'Outlook': 0.24674981977443933, 'Temperature': 0.02922256565895487, 'Humidity': 0.15183550136234164, 'Wind': 0.048127030408269544}


The id3 function stores the information gains of all the attributes, selecting the best node and calling itself recursively to further build the tree.

In [60]:

def id3(data, features, target_attribute_name="Label", parent_node_class=None):

    if len(data[target_attribute_name].unique()) <= 1:
        return data[target_attribute_name].unique()[0]
    elif len(data) == 0:
        return parent_node_class

    elif len(features) == 0:
        return data[target_attribute_name].value_counts().idxmax()

    else:
        parent_node_class = data[target_attribute_name].value_counts().idxmax()

        feature_gains = {feature: information_gain(data, feature, target_attribute_name) for feature in features}

        best_feature = max(feature_gains, key=feature_gains.get)

        tree = {best_feature: {}}

        remaining_features = [feature for feature in features if feature != best_feature]

        for value in data[best_feature].unique():
            subset = data[data[best_feature] == value]

            subtree = id3(subset, remaining_features, target_attribute_name, parent_node_class)

            tree[best_feature][value] = subtree

        return tree

The Tree has been printed below

In [81]:
features = data.drop('Label',axis=1).columns
id3_tree = id3(data, features)
print(id3_tree)

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}


This function is used for making predictions

In [63]:
def predict(instance, tree):

    for attribute, subtree in tree.items():
        value = instance[attribute]
        if value in subtree:
            if isinstance(subtree[value], dict):
                return predict(instance, subtree[value])
            else:
                return subtree[value]

Test instance for prediction has been hard coded and even though the label is being passed as an input parameter, it is not being used for the prediction

In [72]:
test_instance = {'Outlook': 'Overcast', 'Temperature': 'Hot','Humidity':'High','Wind':'Weak','Label':'No'}

prediction = predict(test_instance, id3_tree)
print("Predicted Output:", prediction)

Predicted Output: Yes
